In [18]:
from datetime import datetime, timedelta
import pandas_datareader as pdr
from finta import TA
import pandas as pd


class EquityData():
    
    """       
    A class used to get end of day data 

    Attributes
    ----------
    ticker_list : list
        A list of tickers to get end of day data for
    indicator_list : list
        list of indicators to calculate
        
    Examples
    --------
    # With Params
    ticker_list = ['UBER']
    indicator_list = ['EMA']
    obj = EquityData(ticker_list, indicator_list)
    print(obj.df)

    # Without Params
    obj = EquityData()
    print(obj.df)
    """
    
    def __init__(self, ticker_list=None, indicator_list=None):

        self.start_date = datetime.now().date() - timedelta(days=20)
        self.end_date = datetime.now().date()
        if ticker_list == None:
            self.ticker_list = ['AAPL', 'TSLA']
        else:
            self.ticker_list = ticker_list
        if indicator_list == None:
            self.indicator_list = ['SMA', 'EMA']
        else:
            self.indicator_list = indicator_list
        self.df = pd.DataFrame([])
        self.get_eod_data(self)

    @staticmethod
    def get_eod_data(self):

        """
        Get EOD data for a list of tickers
        """
        eod_df = pd.DataFrame([])

        for t in self.ticker_list:
            results = pdr.DataReader(t, 'yahoo', self.start_date, self.end_date)
            results['ticker'] = t
            eod_df = eod_df.append(results)

        # Calc the list of indicators for each set of EOD data
        eod_df = eod_df.groupby(['ticker']).apply(lambda x: self.compute_analytics(x, self.indicator_list)).reset_index()

        eod_df = self.clean_data(eod_df)
        self.df = eod_df

    @staticmethod
    def compute_analytics(df, indicators):

        """
        Get calculations for each ticker in the df
        """

        for indicator in indicators:
            # Using eval so we can iterate over a list of indicators
            data = eval('TA.' + indicator + '(df)')

            # TA returns a set -> so we convert that to a data frame
            data = data.to_frame()

            # The columns names are too long so I split the string
            # and join that with the indicator name for my indicator column name
            first_string = data.columns[0].split()[0]
            data = data.rename(columns={data.columns[0]: first_string})
            data = data.add_prefix(indicator + '_')

            # Merge data frames based on the date
            df = df.merge(data, left_index=True, right_index=True)

        return df

    @staticmethod
    def clean_data(df):
        
        """
        Update column names to follow PEP8 and sort values
        """
        
        df = df.rename(columns={'Date': 'date', 'High': 'high', 'Low': 'low', 'Open': 'open',
                                'Close': 'close', 'Volume': 'vol', 'Adj Close': 'adj_close',
                                'SMA_41': 'sma_41', 'EMA_9': 'ema_9'})
        df = df.sort_values(by=['ticker', 'date'])
        return df


In [19]:
# Optional Params
# ticker_list = ['UBER']
# indicator_list = ['EMA']
# obj = EquityData(ticker_list, indicator_list)
# print(obj.df)


obj = EquityData()
print(obj.df)

         date        high         low        open       close          vol  \
0  2020-11-02  110.680000  107.320000  109.110001  108.769997  122866900.0   
2  2020-11-03  111.489998  108.730003  109.660004  110.440002  107624400.0   
4  2020-11-04  115.589996  112.349998  114.139999  114.949997  138235500.0   
6  2020-11-05  119.620003  116.870003  117.949997  119.029999  126387100.0   
8  2020-11-06  119.199997  116.129997  118.320000  118.690002  114457900.0   
10 2020-11-09  121.989998  116.050003  120.500000  116.320000  154515300.0   
12 2020-11-10  117.589996  114.129997  115.550003  115.970001  138023400.0   
14 2020-11-11  119.629997  116.440002  117.190002  119.489998  112295000.0   
16 2020-11-12  120.529999  118.570000  119.620003  119.209999  103162300.0   
18 2020-11-13  119.669998  117.870003  119.440002  119.260002   81581900.0   
20 2020-11-16  120.989998  118.150002  118.919998  120.300003   91183000.0   
22 2020-11-17  120.669998  118.959999  119.550003  119.389999   